<a href="https://colab.research.google.com/github/A7medM0sta/SamSum_Summerization/blob/main/SamSum_Summerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB

In [14]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download("punkt")

import os
import zipfile
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
model = "google/pegasus-cnn_dailymail"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# !pip install datasets
from datasets import load_dataset

# Load the Samsung samsum dataset
dataset = load_dataset('Samsung/samsum')


The repository for Samsung/samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Samsung/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [22]:
# dataset_samsum = load_from_disk('samsum_dataset')
# dataset_samsum
dataset_samsum = load_dataset('Samsung/samsum')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [15]:
def download_file():
    if not os.path.exists(local_data_file):
        filename, headers = request.urlretrieve(
        url = source_URL,
        filename = local_data_file)

# The extract_zip_file function should be at the same level as download_file
def extract_zip_file():
    unzip_path = unzip_dir
    os.makedirs(unzip_path, exist_ok = True)
    with zipfile.ZipFile(local_data_file, 'r') as zip_ref:
        zip_ref.extractall(unzip_path)

In [25]:
def validate_all_files_exist():
        try:
            validation_status = None
            all_files = dataset_samsum
            for file in all_files:
                if file not in ["train", "test", "validation"]:
                    validation_status = False
                else:
                    validation_status = True
            return validation_status

        except Exception as e:
            raise e

In [26]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [27]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [28]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [ ]:
%pip install tensorboard
%load_ext tensorboard
%tensorboard --logdir logs

In [38]:
from transformers import TrainingArguments, Trainer
from datasets import load_from_disk # Import load_from_disk

# After executing the mapping function in cell ipython-input-27-9a6514769803
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

# Assign the mapped dataset back to dataset_samsum
dataset_samsum = dataset_samsum_pt.remove_columns(['id', 'dialogue', 'summary'])


trainer_args = TrainingArguments(
        output_dir='pegasus-samsum',
        num_train_epochs=1,
        warmup_steps=500,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        weight_decay=0.01,
        logging_steps=10,
        evaluation_strategy='steps',
        eval_steps=500,
        save_steps=1e6,
        gradient_accumulation_steps=16
  )

trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer=tokenizer,
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum["train"],
            eval_dataset=dataset_samsum["validation"])

trainer.train()

## Save model
model_pegasus.save_pretrained(os.path.join("pegasus-samsum-model")) # Assuming you want to save in the current directory
## Save tokenizer
tokenizer.save_pretrained(os.path.join("tokenizer")) # Assuming you want to save in the current directory

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,1.659900,1.483296


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [45]:
def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """
        split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements.
        """

        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                            batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                            column_text="article",
                            column_summary="highlights"):
    article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                        attention_mask=inputs["attention_mask"].to(device),
                        length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
            for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

def evaluation(self):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
    model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

    #loading data
    dataset_samsum_pt = load_from_disk(self.config.data_path)
    rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    rouge_metric = load_metric('rouge')
    score = self.calculate_metric_on_test_ds(
    dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary')

    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

    return rouge_dict

In [47]:
from transformers import AutoTokenizer
from transformers import pipeline

def predict(text):
    tokenizer = AutoTokenizer.from_pretrained('/content/tokenizer')
    model = '/content/pegasus-samsum-model'

    kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

    Pipeline = pipeline("summarization", model= model, tokenizer=tokenizer)

    summary = Pipeline(text, **kwargs)[0]["summary_text"]
    return summary

In [48]:
predict(
    "US-based private equity firm General Atlantic is in talks to invest about \
    $850 million to $950 million in Reliance Industries' digital unit Jio \
    Platforms, the Bloomberg reported. Saudi Arabia's $320 billion sovereign \
    wealth fund is reportedly also exploring a potential investment in the \
    Mukesh Ambani-led company. The 'Public Investment Fund' is looking to \
    acquire a minority stake in Jio Platforms."
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 128, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)


"General Atlantic is in talks to invest $850 million to $950 million in Reliance Industries' digital unit Jio Platforms. Saudi Arabia's $320 billion sovereign wealth fund is also exploring a potential investment in the Mukesh Ambani-led company."

In [51]:
model_save_path = '/content/drive/My Drive/my_pegasus_model/pegasus-samsum-model'
part_save_path = '/content/drive/My Drive/my_pegasus_model/pegasus-samsum'
tokenizer_save_path = '/content/drive/My Drive/my_pegasus_model/tokenizer'

In [53]:
## Save model
model_pegasus.save_pretrained(model_save_path)
## Save tokenizer
tokenizer.save_pretrained(tokenizer_save_path)
# part_save_path.save_pretrained(part_save_path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('/content/drive/My Drive/my_pegasus_model/tokenizer/tokenizer_config.json',
 '/content/drive/My Drive/my_pegasus_model/tokenizer/special_tokens_map.json',
 '/content/drive/My Drive/my_pegasus_model/tokenizer/spiece.model',
 '/content/drive/My Drive/my_pegasus_model/tokenizer/added_tokens.json',
 '/content/drive/My Drive/my_pegasus_model/tokenizer/tokenizer.json')